In [1]:
import numpy as np
import pandas as pd

In [2]:
data_url = 'https://raw.githubusercontent.com/jkropko/contrans/main/examples/ASA%20All%20NBA%20Raw%20Data.csv'
nba = pd.read_csv(data_url)

In [3]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg', 'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP',
       'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_s

In [7]:
nba = nba[['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm']]

In [9]:
pd.set_option('display.max_rows', 42)
pd.set_option('display.max_columns', 42)
nba.head(3)

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Inactives,Opponent_Abbrev,player,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,off_rtg,def_rtg,bpm
0,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Tyrese Haliburton,halibty01,1,39:28,7,14,0.500,2,5,0.400,1,1,1.00,2,2,4,10,2,0,1,0,17,-9,0,0,137,132,1.7
1,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Buddy Hield,hieldbu01,1,35:53,8,23,0.348,5,14,0.357,0,0,0.00,0,3,3,6,3,0,2,3,21,0,0,0,94,128,-2.3
2,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Oshae Brissett,brissos01,1,35:47,10,20,0.500,5,10,0.500,3,4,0.75,3,5,8,3,1,0,0,5,28,-9,0,0,137,133,4.4


## First normal form

- Every table has a primary key? Yes.

- Non-atomic data?

In [10]:
nba = nba.drop(['Inactives'], axis=1)
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


 - Repeating groups? No.
 
 - First normal form!

## Second normal form

- Data in first normal form? Yes. 

- Every non-prime column must depend on the ENTIRE primary key (gameid + playerid) and not just part of the primary key

In [11]:
nba['game_player_id'] = nba['game_id'] + '_' + nba['player_id']

In [12]:
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


## Third normal form

- Data must be in second normal form

- Calculated column:

fg_pct
fg3_pct
ft_pct

- Transitive dependencies:

Some columns depend on player

Some columns depend on game

some columns depend on team

Each of these will get a separate table
